In [1]:
import os
import pandas as pd
import re
Already_Processed_files_path = 'D:\\faraz_target\Already_Processed_files.txt'
staging_path = 'D:\\faraz'
target_path='D:\\faraz_target\\'
address_path='D:\\faraz_add\Areas_in_blore.xlsx'
df_address=pd.read_excel(address_path)

In [2]:
processed_csv_files = []
source_csv_files = []
##Check if it’s a new file or not. We do not want to reprocess already processed
##files.
def file_check_module():
    global Unprocessed_files
    ##checking if File extensions is csv
    for name in os.listdir(staging_path):
        if name.endswith(".csv"):
            source_csv_files.append(name)
    
    processed = open(Already_Processed_files_path, 'r')
    processed_files = processed.readlines()
    processed.close()
    for file in processed_files:
        processed_csv_files.append(file)
    print(processed_csv_files)
    ##Check if it’s a new file or not
    Unprocessed_files =list(set(source_csv_files) - set(processed_csv_files))
    print(Unprocessed_files)
    print("Type of Unprocessed_files:", type(Unprocessed_files))
    global Unprocessed_non_empty_file
    Unprocessed_non_empty_file=[]
    for Unprocessed_file in Unprocessed_files:
        full_path=staging_path + "\\" + Unprocessed_file
        print(full_path)
        df = pd.read_csv(full_path)
        print(df.empty)
        ##checking for a non-empty file.
        if not(df.empty):
            Unprocessed_non_empty_file.append(full_path)

In [3]:
file_check_module()

[]
['data_file_20210528182844.csv', 'data_file_20210528182864.csv', 'data_file_20210528182554.csv', 'data_file_20210527182730.csv']
Type of Unprocessed_files: <class 'list'>
D:\faraz\data_file_20210528182844.csv
False
D:\faraz\data_file_20210528182864.csv
True
D:\faraz\data_file_20210528182554.csv
False
D:\faraz\data_file_20210527182730.csv
False


In [4]:
##removing commas, period, exclamations or any other special/junk characters etc.
def Replace_Special_Characters(input):
    process = re.sub('[^A-Za-z0-9]+', ' ', input)
    return process

In [5]:
#removing commas, period, exclamations or any other special/junk characters etc.
def Replace_Special_Characters_review(input):
    process = re.sub('[^A-Za-z0-9]+', ' ', input)
    process = re.sub('0 RATED n', '', process)
    process = re.sub('  ', ' ', process)
    return process

In [6]:
#Removing preceding Plus
def Phone_Number_Validation(input):
    process = input.lstrip("+")
    return process

In [7]:
##My data quality check
#We can have checks to remove words like below .
##you've -> you have
##he's -> he is
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [8]:
def data_quality_check():
    for to_process in Unprocessed_non_empty_file:
        target_file_name = os.path.basename(to_process)
        target_file_bad = re.sub('.csv','.bad',target_file_name)
        target_file_out = re.sub('.csv','.out',target_file_name)
        target_file_metadata = re.sub('.csv','.metadata',target_file_name)
        target_full_path_bad = target_path + target_file_bad
        target_full_path_out = target_path + target_file_out
        target_full_path_meta = target_path + target_file_metadata
        print("target path is ::",target_full_path_out)
        df = pd.read_csv(to_process)
        df['address']= df['address'].map(Replace_Special_Characters)
        df['reviews_list']= df['reviews_list'].map(Replace_Special_Characters_review)
        df['reviews_list']= df['reviews_list'].map(decontracted)
        ##Splitting Phone column
        df[['phone_1', 'phone_2']] = df['phone'].str.split('\r\n', n=1, expand=True)
        df['phone_1']= df['phone_1'].astype(str).map(Phone_Number_Validation)
        df['phone_2']= df['phone_2'].astype(str).map(Phone_Number_Validation)
        df_name_is_null = df.loc[df['name'].isnull()]
        df_phone = df.loc[df['phone_1'].isnull()]
        df_location = df.loc[df['location'].isnull()]
        df = df.loc[df['location'].notnull()]
        df = df.loc[df['phone_1'].notnull()]
        df = df.loc[df['location'].notnull()]
        ##Validation of location field for correctness of data by looking up to the Areas_in_blore.csv file
        df = df.assign(InDF_address=df.location.isin(df_address.Area).astype(int))
        df_out = df.loc[df['InDF_address'] == 1]
        df_bad = df.loc[df['InDF_address'] == 0]
        df_bad_dropped_InDF_address = df_bad
        df_bad.index.names = ['Row_num_list']
        df_out.drop(['phone', 'InDF_address'], axis=1, inplace=True)
        if(df_bad.shape[0] > 0):
            df_bad_dropped_InDF_address.to_csv(target_full_path_bad,index=0)
            df_bad.loc[df_bad['InDF_address'] == 0, 'Type_of_issue'] = "Invalid_Address"
            df_bad['Type_of_issue'].to_csv(target_full_path_meta)
        ##Checks for null values in fields 
        if(df_location.shape[0] > 0):
            ##Capturing all the bad records to a .bad & metadata file.
            df_location.to_csv(target_full_path_bad, mode='a', header=False,index=1)
            print("Bad data appended at the location::",target_full_path_bad)
            df_location['Type_of_issue']="location is null"
            df_location['Type_of_issue'].to_csv(target_full_path_meta, mode='a', header=False,index=1)
        ##Checks for null values in fields 
        if(df_phone.shape[0] > 0):
            ##Capturing all the bad records to a .bad & metadata file.
            df_phone.to_csv(target_full_path_bad, mode='a', header=False,index=1)
            print("Bad data appended at the location::",target_full_path_bad)
            df_phone['Type_of_issue']="phone number is null"
            df_phone['Type_of_issue'].to_csv(target_full_path_meta, mode='a', header=False,index=1)
        ##Checks for null values in fields 
        if(df_name_is_null.shape[0] > 0):
            ##Capturing all the bad records to a .bad & metadata file.
            df_name_is_null.to_csv(target_full_path_bad, mode='a', header=False,index=1)
            print("Bad data appended at the location::",target_full_path_bad)
            df_name_is_null['Type_of_issue']="name is null"
            df_name_is_null['Type_of_issue'].to_csv(target_full_path_meta, mode='a', header=False,index=1)
        ##Capturing all the clean records to a .out file.
        df_out.to_csv(target_full_path_out,index=0)
        mark_processed = open(Already_Processed_files_path,"a")
        mark_processed.writelines(target_file_name)
        mark_processed.write("\n")
        mark_processed.close()
        print(to_process)
        print(df.shape)

In [9]:
data_quality_check()

target path is :: D:\faraz_target\data_file_20210528182844.out


C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Bad data appended at the location:: D:\faraz_target\data_file_20210528182844.bad
D:\faraz\data_file_20210528182844.csv
(7378, 20)
target path is :: D:\faraz_target\data_file_20210528182554.out


C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Bad data appended at the location:: D:\faraz_target\data_file_20210528182554.bad
D:\faraz\data_file_20210528182554.csv
(7376, 20)
target path is :: D:\faraz_target\data_file_20210527182730.out


C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Faraz Khan\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Bad data appended at the location:: D:\faraz_target\data_file_20210527182730.bad
D:\faraz\data_file_20210527182730.csv
(7378, 20)
